In [ ]:
import utils
utils.create_unique_folder("test")
utils.create_unique_folder("test")


In [ ]:
from utils import load_model
from TCN import TCNAE

path = "HyperparameterSearchAllModels/Models/TCNAE_472.pth"
paras = {"seq_len": 61, "input_dim": 1, "num_channels": [8, 16], "kernel_size": 7, "latent_dim": 10}
model = load_model(path, TCNAE, paras)
model

In [ ]:
from utils import load_np_dataset
import torch
import numpy as np
fps = 90
cond = "nod"
f = 62
b = 500
a = 200
angles_correct = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
angles_incorrect = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
names_correct = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
names_incorrect = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
pat_names = np.unique(names_correct)
n = int(0.7 * len(pat_names))
train_pats = pat_names[:n]
test_pats = pat_names[n:]
train_data = angles_correct[np.isin(names_correct, train_pats)]
test_correct = angles_correct[np.isin(names_correct, test_pats)]
test_incorrect = angles_incorrect[np.isin(names_incorrect, test_pats)]

test_correct = torch.tensor(test_correct).type(torch.float)[:, None, :]
test_incorrect = torch.tensor(test_incorrect).type(torch.float)[:, None, :]
train_correct= torch.tensor(train_data).type(torch.float)[:, None, :]
model.eval()
with torch.no_grad():
    train_correct_out = model(train_correct)
    correct_out = model(test_correct)
    incorrect_out = model(test_incorrect)
    mse_correct = torch.mean((correct_out - test_correct)**2, dim=(1, 2))
    mse_incorrect = torch.mean((incorrect_out - test_incorrect)**2, dim=(1, 2))
    mse_train_correct = torch.mean((train_correct_out - train_correct)**2, dim=(1, 2))
    print(mse_correct)
    print(mse_incorrect)
    print(mse_train_correct)

In [ ]:
import matplotlib.pyplot as plt
perc = np.linspace(80, 100, 100)
ths = np.percentile(mse_train_correct.cpu().numpy(), perc)
fig, ax = plt.subplots()
res = {"TH_number": list(), "TH": list(), "Correct": list(), "Incorrect": list()}
for p, th in zip(perc, ths):
    correct_acc = np.mean((mse_correct < th).cpu().numpy())
    incorrect_acc = np.mean((mse_incorrect > th).cpu().numpy())
    res["TH_number"].append(th)
    res["TH"].append(p)
    res["Correct"].append(correct_acc)
    res["Incorrect"].append(incorrect_acc)

ax.plot(res["TH"], res["Correct"], label="Correct", color="b")
ax.plot(res["TH"], res["Incorrect"], label="Incorrect", color="orange")
ax.axhline(0.9, linestyle="--", color="red", alpha=0.4)
ax.set_ylabel("Recall")
ax.set_xlabel("Percentile")
ax.legend()
plt.show()

In [ ]:
from utils import load_model
from TCN import TCNAE
import torch
import numpy as np

model_parameter = {"seq_len":43, "input_dim":1, "num_channels": [8, 16], "kernel_size":3, "latent_dim":10}
model = load_model("Results/LOSO_500msSeq/Models/TCNAE_12_nod.pth", TCNAE, model_parameter)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

fps = 90
cond = "nod"
f = 44
b = 300
a = 200
angles_correct = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
angles_incorrect = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
names_correct = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
names_incorrect = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
pat_names = np.unique(names_correct)
n = int(0.7 * len(pat_names))
train_pats = pat_names[:n]
test_pats = pat_names[n:]
train_data = angles_correct[np.isin(names_correct, train_pats)]
test_correct = angles_correct[np.isin(names_correct, test_pats)]
test_incorrect = angles_incorrect[np.isin(names_incorrect, test_pats)]

test_correct = torch.tensor(test_correct).type(torch.float)[:, None, :]
test_incorrect = torch.tensor(test_incorrect).type(torch.float)[:, None, :]
train_correct= torch.tensor(train_data).type(torch.float)[:, None, :]
model.eval()
with torch.no_grad():
    train_correct_out = model(train_correct)
    correct_out = model(test_correct)
    incorrect_out = model(test_incorrect)
    mse_correct = torch.mean((correct_out - test_correct)**2, dim=(1, 2))
    mse_incorrect = torch.mean((incorrect_out - test_incorrect)**2, dim=(1, 2))
    mse_train_correct = torch.mean((train_correct_out - train_correct)**2, dim=(1, 2))
    print(mse_correct)
    print(mse_incorrect)
    print(mse_train_correct)

In [ ]:
import matplotlib.pyplot as plt
trial_nr = 1
correct = {"in": test_correct[trial_nr][0].numpy(), "out": correct_out[trial_nr][0].numpy()}
incorrect = {"in": test_incorrect[trial_nr][0].numpy(), "out": incorrect_out[trial_nr][0].numpy()}

fig, (ax_correct, ax_incorrect) = plt.subplots(2, sharex=True)
ax_correct.plot(correct["in"], color="blue")
ax_correct.set_title("Correct")
ax_incorrect.plot(incorrect["in"], color="blue")
ax_incorrect.set_title("Incorrect")
fig.tight_layout()
plt.show()

mse_correct = np.mean((correct["in"] - correct["out"])**2)
mse_incorrect = np.mean((incorrect["in"] - incorrect["out"])**2)

fig, (ax_correct, ax_incorrect) = plt.subplots(2, sharex=True)
ax_correct.plot(correct["out"], color="blue")
ax_correct.set_title(f"Reconstructed correct") #; MSE {mse_correct}")
ax_incorrect.plot(incorrect["out"], color="blue")
ax_incorrect.set_title(f"Reconstructed incorrect") #; MSE {mse_incorrect}")
fig.tight_layout()
plt.show()
print("Correct MSE: ", mse_correct)
print("Incorrect MSE: ", mse_incorrect)
# def create_plot_auto(correct, incorrect):
#     fig, (ax_correct, ax_incorrect) = plt.subplot(2, sharex=True)
#     ax_correct.plot(correct)

# Test Different algos

In [10]:
# Cross Validation

import numpy as np
from sklearn.model_selection import KFold, ParameterGrid
from tqdm import tqdm


def get_opt_clf(clf, params, correct_samples, incorrect_samples, names_correct, names_incorrect, pats, print_results=True):
    kf = KFold(n_splits=5)
    best_params = None
    best_score = -np.inf
    best_results = None
    counter_params = np.prod([len(param) for param in params.values()])
    
    progress_bar = tqdm(total=counter_params, desc="Progress")

    for param in ParameterGrid(params):
        model = clf(**param)
        scores = list()
        incorrect_scores = list()
        correct_scores = list()

        for train_index, test_index in kf.split(pats):
            train_pats, test_pats = pats[train_index], pats[test_index]
            X_train = correct_samples[np.isin(names_correct, train_pats)]
            model.fit(X_train)
            X_correct_test = correct_samples[np.isin(names_correct, test_pats)]
            X_incorrect_test = incorrect_samples[np.isin(names_incorrect, test_pats)]

            correct_score = model.predict(X_correct_test)
            incorrect_score = model.predict(X_incorrect_test)
            correct_score = (correct_score == 1).mean()
            incorrect_score = (incorrect_score == -1).mean()
            scores.append((correct_score+incorrect_score)/2)
            incorrect_scores.append(incorrect_score)
            correct_scores.append(correct_score)
        
        mean_score = np.mean(scores)
        if mean_score > best_score:
            best_score = mean_score
            best_params = param
            best_results = {"correct": correct_scores, "incorrect": incorrect_scores}
        
        progress_bar.postfix = f"Best Score: {best_score:.3f}, Correct: {np.mean(best_results['correct']):.3f}, Incorrect: {np.mean(best_results['incorrect']):.3f}"
        progress_bar.update(1)

    if print_results:
        print()
        print("CV Results:")
        print(f"    Best parameters: {best_params}")
        print(f"    Best score: {best_score}")
        print(f"    Correct: {np.mean(best_results['correct'])}")
        print(f"    Incorrect: {np.mean(best_results['incorrect'])}")
        print()

    return best_params, best_score, best_results

 

In [9]:
# overall function of method
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json

def test_method(method, param_grid, fps=90, f=44, b=300, a=200):

    all_res = dict()

    for cond in ["gaze", "headAndGaze", "nod"]:
        print("="*80)
        print(f"Condition: {cond}")
        print("="*80)
        angles_correct = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
        angles_incorrect = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
        names_correct = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
        names_incorrect = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
        pat_names = np.unique(names_correct)
        n = int(0.7 * len(pat_names))
        train_pats = pat_names[:n]
        test_pats = pat_names[n:]
        train_data = angles_correct[np.isin(names_correct, train_pats)]
        test_correct = angles_correct[np.isin(names_correct, test_pats)]
        test_incorrect = angles_incorrect[np.isin(names_incorrect, test_pats)]


        # n_estimators: 100, 200, 300 (default: 100)
        # max_samples: Number of samples to draw from X to train each base estimator
        #       'auto', 0.5, 0.75 (default: 'auto')
        # max_features: Number of features to draw from X to train each base estimator
        #       0.1, 0.5, 0.75, 1.0 (default: 1.0)
        # contamination: Proportion of outliers in the data set
        #       'auto', 0.05, 0.1, 0.2 (default: 'auto')


        best_params, best_score, best_results = get_opt_clf(method, param_grid, angles_correct, angles_incorrect, names_correct, names_incorrect, train_pats)




        clf = method(**best_params).fit(train_data)

        # clf = IsolationForest(max_features=1.0, random_state=42).fit(train_data)

        # model = IsolationForest()
        # param_grid = {
        #     "n_estimators": [100, 200, 300],
        #     "max_samples": ['auto', 0.5, 0.75],
        #     "max_features": [0.1, 0.5, 0.75, 1.0],
        #     "contamination": ['auto', 0.05, 0.1, 0.2]   
        # }

        # clf = GridSearchCV(model, param_grid)


        correct_scores = clf.predict(test_correct)
        incorrect_scores = clf.predict(test_incorrect)

        correct_acc = (correct_scores==1).mean()
        incorrect_acc = (incorrect_scores==-1).mean()

        print("Correct", (correct_scores==1).mean(), "Incorrect", (incorrect_scores==-1).mean())

        sns.histplot({"Correct": correct_scores, "Incorrect": incorrect_scores},
                    multiple="layer", common_norm=False, stat="percent")
        plt.show()

        all_res[cond] = {
            "best_params": best_params,
            "best_score": best_score,
            "best_results": best_results,
            "correct_test": correct_acc,
            "incorrect_test": incorrect_acc,
            "macro_test": (correct_acc + incorrect_acc) / 2,
            "micro_test": (correct_acc * len(correct_scores) + incorrect_acc * len(incorrect_scores)) / (len(correct_scores) + len(incorrect_scores))
        }

        print()

    with open(f"Results/ml_algos_res/{method.__name__}.json", "w") as f:
        json.dump(all_res, f, indent=4) 

## Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest

param_grid = {
        "n_estimators": [100, 200, 300],
        "max_samples": ['auto', 0.5, 0.75],
        "max_features": [0.1, 0.5, 0.75, 1.0],
        "contamination": ['auto', 0.05, 0.1, 0.2]   
    }

test_method(IsolationForest, param_grid)

In [ ]:
# from sklearn.ensemble import IsolationForest
# import matplotlib.pyplot as plt
# import seaborn as sns
# from utils import load_np_dataset
# from sklearn.model_selection import GridSearchCV
# import numpy as np
# fps = 90
# cond = "nod"
# f = 44
# b = 300
# a = 200
# all_res = dict()

# for cond in ["gaze", "headAndGaze", "nod"]:
#     print("="*80)
#     print(f"Condition: {cond}")
#     print("="*80)
#     angles_correct = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
#     angles_incorrect = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
#     names_correct = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
#     names_incorrect = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
#     pat_names = np.unique(names_correct)
#     n = int(0.7 * len(pat_names))
#     train_pats = pat_names[:n]
#     test_pats = pat_names[n:]
#     train_data = angles_correct[np.isin(names_correct, train_pats)]
#     test_correct = angles_correct[np.isin(names_correct, test_pats)]
#     test_incorrect = angles_incorrect[np.isin(names_incorrect, test_pats)]


#     # n_estimators: 100, 200, 300 (default: 100)
#     # max_samples: Number of samples to draw from X to train each base estimator
#     #       'auto', 0.5, 0.75 (default: 'auto')
#     # max_features: Number of features to draw from X to train each base estimator
#     #       0.1, 0.5, 0.75, 1.0 (default: 1.0)
#     # contamination: Proportion of outliers in the data set
#     #       'auto', 0.05, 0.1, 0.2 (default: 'auto')

#     param_grid = {
#         "n_estimators": [100, 200, 300],
#         "max_samples": ['auto', 0.5, 0.75],
#         "max_features": [0.1, 0.5, 0.75, 1.0],
#         "contamination": ['auto', 0.05, 0.1, 0.2]   
#     }

#     for k in param_grid.keys():
#         all_res[k] = list()

#     best_params, best_score, best_results = get_opt_clf(IsolationForest, param_grid, angles_correct, angles_incorrect, names_correct, names_incorrect, train_pats)




#     clf = IsolationForest(**best_params).fit(train_data)

#     # clf = IsolationForest(max_features=1.0, random_state=42).fit(train_data)

#     # model = IsolationForest()
#     # param_grid = {
#     #     "n_estimators": [100, 200, 300],
#     #     "max_samples": ['auto', 0.5, 0.75],
#     #     "max_features": [0.1, 0.5, 0.75, 1.0],
#     #     "contamination": ['auto', 0.05, 0.1, 0.2]   
#     # }

#     # clf = GridSearchCV(model, param_grid)


#     correct_scores = clf.predict(test_correct)
#     incorrect_scores = clf.predict(test_incorrect)

#     correct_acc = (correct_scores==1).mean()
#     incorrect_acc = (incorrect_scores==-1).mean()

#     print("Correct", (correct_scores==1).mean(), "Incorrect", (incorrect_scores==-1).mean())

#     sns.histplot({"Correct": correct_scores, "Incorrect": incorrect_scores},
#                 multiple="layer", common_norm=False, stat="percent")
#     plt.show()

#     all_res[cond] = {
#         "best_params": best_params,
#         "best_score": best_score,
#         "best_results": best_results,
#         "correct_test": correct_acc,
#         "incorrect_test": incorrect_acc,
#         "macro_test": (correct_acc + incorrect_acc) / 2,
#         "micro_test": (correct_acc * len(correct_scores) + incorrect_acc * len(incorrect_scores)) / (len(correct_scores) + len(incorrect_scores))
#     }

#     print()

# import json
# with open("Results/ml_algos_res/IsolationForest.json", "w") as f:
#     json.dump(all_res, f, indent=4) 

## One Class SVM

In [ ]:
from sklearn.svm import OneClassSVM

param_grid = {
        "nu": [0.1, 0.2, 0.3, 0.4, 0.5],
        "kernel": ["linear", "rbf", "poly", "sigmoid"],
        "gamma": ['scale', 'auto']
    }

test_method(OneClassSVM, param_grid)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm

fps = 90
cond = "gaze"
f = 44
b = 300
a = 200

for cond in ["gaze", "headAndGaze", "nod"]:#
    print("="*80)
    print(f"Condition: {cond}")
    print("="*80)
    angles_correct = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
    angles_incorrect = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
    names_correct = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
    names_incorrect = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
    pat_names = np.unique(names_correct)
    n = int(0.7 * len(pat_names))
    train_pats = pat_names[:n]
    test_pats = pat_names[n:]
    train_data = angles_correct[np.isin(names_correct, train_pats)]
    test_correct = angles_correct[np.isin(names_correct, test_pats)]
    test_incorrect = angles_incorrect[np.isin(names_incorrect, test_pats)]

    # kernel: "linear", "rbf", "poly", "sigmoid", "precomputed"
    # nu: An upper bound on the fraction of training errors and a lower bound of the fraction of support vectors. 
    #     Should be in the interval (0, 1]. By default 0.5 will be taken.
    # gamma: gamma is a hyperparameter that we have to set before the training model.
    #        - 'scale': uses 1 / (n_features * X.var()) as value of gamma
    #        - 'auto': uses 1 / n_features.
    #        - float > 0
    param_grid = {
        "nu": [0.1, 0.2, 0.3, 0.4, 0.5],
        "kernel": ["linear", "rbf", "poly", "sigmoid"],
        "gamma": ['scale', 'auto']
    }

    best_params, best_score, best_results = get_opt_clf(svm.OneClassSVM, param_grid, angles_correct, angles_incorrect, names_correct, names_incorrect, train_pats)


    clf = svm.OneClassSVM(**best_params)
    clf.fit(train_data)

    correct_scores = clf.predict(test_correct)
    incorrect_scores = clf.predict(test_incorrect)

    print("Correct", (correct_scores==1).mean(), "Incorrect", (incorrect_scores==-1).mean())

    sns.histplot({"Correct": correct_scores, "Incorrect": incorrect_scores},
                multiple="layer", common_norm=False, stat="percent")
    plt.show()
    print()

## Local outliere factor

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import LocalOutlierFactor

fps = 90
cond = "gaze"
f = 44
b = 300
a = 200

for gaze in ["gaze", "headAndGaze", "nod"]:
    angles_correct = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
    angles_incorrect = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
    names_correct = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
    names_incorrect = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
    pat_names = np.unique(names_correct)
    n = int(0.7 * len(pat_names))
    train_pats = pat_names[:n]
    test_pats = pat_names[n:]
    train_data = angles_correct[np.isin(names_correct, train_pats)]
    test_correct = angles_correct[np.isin(names_correct, test_pats)]
    test_incorrect = angles_incorrect[np.isin(names_incorrect, test_pats)]

    # n_neighbors: Number of neighbors to use by default for kneighbors queries.
    #      5, 10, 20, 50 (default: 20)
    # algorithm: Algorithm used to compute the nearest neighbors
    #     'auto', 'ball_tree', 'kd_tree', 'brute' (default: 'auto')
    # leaf_size: Leaf size passed to BallTree or KDTree.
    #     10, 20, 30, 40, 50 (default: 30)
    # metric: metric used for the distance computation
    #     'euclidean', 'manhattan', 'minkowski', 'chebyshev' (default: 'minkowski')

    param_grid = {
        "n_neighbors": [5, 10, 20, 50],
        "algorithm": ['auto', 'ball_tree', 'kd_tree', 'brute'],
        "leaf_size": [10, 20, 30, 40, 50],
        "metric": ['euclidean', 'manhattan', 'minkowski', 'chebyshev'],
        'novelty': [True]
    }

    best_params, best_score, best_results = get_opt_clf(LocalOutlierFactor, param_grid, angles_correct, angles_incorrect, names_correct, names_incorrect, train_pats)


    clf = LocalOutlierFactor(**best_params)
    clf.fit(train_data)

    correct_scores = clf.predict(test_correct)
    incorrect_scores = clf.predict(test_incorrect)

    print("Correct", (correct_scores==1).mean(), "Incorrect", (incorrect_scores==-1).mean())

    sns.histplot({"Correct": correct_scores, "Incorrect": incorrect_scores},
                multiple="layer", common_norm=False, stat="percent")
    plt.show()
    print()

## Elliptic Envolpe

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.covariance import EllipticEnvelope

fps = 90
cond = "gaze"
f = 44
b = 300
a = 200
angles_correct = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
angles_incorrect = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
names_correct = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
names_incorrect = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
pat_names = np.unique(names_correct)
n = int(0.7 * len(pat_names))
train_pats = pat_names[:n]
test_pats = pat_names[n:]
train_data = angles_correct[np.isin(names_correct, train_pats)]
test_correct = angles_correct[np.isin(names_correct, test_pats)]
test_incorrect = angles_incorrect[np.isin(names_incorrect, test_pats)]

# support_fraction: The proportion of points to be included in the support of the raw MCD estimate.
#      if None, the minimum value of support_fraction will be used within the algorithm: [n_samples + n_features + 1] / 2
#      0.1, 0.2, 0.3, 0.4, 0.5 (default: None), Range: (0, 1)
# contamination: The amount of contamination of the data set, i.e. the proportion of outliers in the data set.
#      0.05, 0.1, 0.2 (default: 0.1), Range: (0, 0.5)

clf = EllipticEnvelope(contamination=0.1)
clf.fit(train_data)

correct_scores = clf.predict(test_correct)
incorrect_scores = clf.predict(test_incorrect)

print("Correct", (correct_scores==1).mean(), "Incorrect", (incorrect_scores==-1).mean())

sns.histplot({"Correct": correct_scores, "Incorrect": incorrect_scores},
             multiple="layer", common_norm=False, stat="percent")
plt.show()

## DBSCAN

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN

fps = 90
cond = "gaze"
f = 44
b = 300
a = 200
angles_correct = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
angles_incorrect = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
names_correct = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
names_incorrect = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
pat_names = np.unique(names_correct)
n = int(0.7 * len(pat_names))
train_pats = pat_names[:n]
test_pats = pat_names[n:]
train_data = angles_correct[np.isin(names_correct, train_pats)]
test_correct = angles_correct[np.isin(names_correct, test_pats)]
test_incorrect = angles_incorrect[np.isin(names_incorrect, test_pats)]

clf = DBSCAN(eps=3, min_samples=10)
# clf.fit(train_data)

correct_scores = clf.fit_predict(test_correct)
incorrect_scores = clf.fit_predict(test_incorrect)

print("Correct", (correct_scores==1).mean(), "Incorrect", (incorrect_scores==-1).mean())

sns.histplot({"Correct": correct_scores, "Incorrect": incorrect_scores},
             multiple="layer", common_norm=False, stat="percent")
plt.show()

In [ ]:
print("Correct", len(angles_correct)/len(pat_names), "Incorrect", len(angles_incorrect)/len(pat_names))

In [2]:
import json
import torch.nn as nn

test_dict = {
    "name": "bla",
    "res": 0.890709001,
    "paras": {
        "num_channels": [8, 16],
        "kernel_size": 5,
        "latent_dim": 16
    },
    "train": {
        "num_epochs": 100,
        "batch_size": 1000,
        "criterion": str(nn.MSELoss())
    },
    "th_perc": 95,
    "th_value": 0.335
}

with open("example.json", "w") as outfile:
    json.dump(test_dict, outfile)


In [ ]:
with open("example.json") as json_file:
    data = json.load(json_file)
    print(type(data))
    print(data)

In [ ]:
te = [[int, {'x': 1, 'y': 2}], [int, {'x': 1, 'y': 2}], [int, {'x': 1, 'y': 2}]]

te = [x + [y] for y in [95, 99] for x in te]
te

In [ ]:
import os
os.cpu_count()

# Load Model

In [ ]:
import os
import torch
from TCN import TCNAE
import utils
import json
import torch.onnx

path = "Results/LOSO_a200_b300_f44_seq43/Models"
model_name = "TCNAE_4"
conditions = ["nod", "gaze", "headAndGaze"]
model_names = ["TCNAE_1_gaze", "TCNAE_1_headAndGaze", "TCNAE_1_nod"]
cond = conditions[0]
# for cond in conditions:
#     with open(os.path.join(path, model_name + f"_{cond}_info.json")) as json_file:
#         model_info = json.load(json_file)
        # print(model_info)
    # model = utils.load_model(os.path.join(path, model_name + f"_{cond}.pth"), TCNAE, model_info["model_parameter"])

for model_name in model_names:
    with open(os.path.join(path, model_name + "_info.json")) as json_file:
        model_info = json.load(json_file)

    model = utils.load_model(os.path.join(path, model_name + ".pth"), TCNAE, model_info["model_parameter"])

    dummy_input = torch.randn(1, 1, model_info["model_parameter"]["seq_len"])
    model.eval()
    # dummy_output = model(dummy_input)
    # print(dummy_output)

    # save model as onnx at the same location like the model_info

    torch.onnx.export(model, dummy_input, os.path.join(path, model_name + ".onnx"))

    print(f"Model has been saved as {model_name}.onnx")

In [ ]:
# Test models
input = torch.randn(5, 1, 1, 1, 1, 1, 1, 61)
output = model(input)

print(output)